In [5]:
import os
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# import matplotlib.pyplot as plt
# import warnings
# import sys
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
# from genericpath import isfile
from TrafficLabelling.utils import *


dataset = 'lycos-ids2017/' # TrafficLabelling lycos-ids2017
modo = 'pca_9' # cos pca_9
segunda = 'knn' # knn mlp
f = 'CPd_dest' # in ['CPdId','CPeIe','CPd_dest','C']:
if dataset == 'lycos-ids2017/':
    dataset_clean = pd.read_csv(dataset + 'dataset_clean_exp_atrib.csv', index_col=[0])
    dataset_clean = pd.get_dummies(dataset_clean, columns=['ip_prot'],drop_first=True)

    # Comentar caso use Pd = 6
    dataset_clean_cat = pd.read_csv(dataset + 'pd4.csv', index_col=[0])
    dataset_clean = pd.concat([dataset_clean_cat, dataset_clean.loc[:,np.invert(dataset_clean.columns.str.contains('[sd]port'))]], axis=1)
    del dataset_clean_cat

    Id = dataset_clean.columns.str.contains('[sd]addr')
    Id = dataset_clean.columns[Id].to_list()
    Pd = dataset_clean.columns.str.contains('[sd]port')
    Pd = dataset_clean.columns[Pd].to_list()

    Ie = dataset_clean.columns.str.match('(src_addr)|(dst_addr)')
    Ie = dataset_clean.columns[Ie].to_list()
    Pe = dataset_clean.columns.str.match('(src_port)|(dst_port)')
    Pe = dataset_clean.columns[Pe].to_list()

    C = np.setdiff1d(dataset_clean.columns,Id+Pd+Ie+Pe+['label','flow_id','timestamp']).tolist()
    Cs = ['flow_duration', 'down_up_ratio', 'pkt_len_max', 'bytes_per_s', 'pkt_per_s', 'fwd_pkt_cnt', 'fwd_pkt_len_tot', 'fwd_pkt_len_max'] + dataset_clean.columns[dataset_clean.columns.str.contains('prot')].to_list()

    # comb_f = {'CPe':C+Pe,'CPd'+str(int(len(Pd)/2)):C+Pd,'PeIe':Pe+Ie,'Pd'+str(int(len(Pd)/2))+'Ie':Pd+Ie,'Pd'+str(int(len(Pd)/2))+'Id'+str(int(len(Id)/2)):Pd+Id,'CPd'+str(int(len(Pd)/2))+'Ie':C+Pd+Ie,'CPd'+str(int(len(Pd)/2))+'Id'+str(int(len(Id)/2)):C+Pd+Id,'C':C,'CPeIe':C+Pe+Ie,'CPd'+str(int(len(Pd)/2))+'_dest':C+Pd[-int(len(Pd)/2):],'CPe_dest':C+Pe[-6:],'Cs':Cs,'CsPd'+str(int(len(Pd)/2))+'_dest':Cs+Pd[-int(len(Pd)/2):]}
    comb_f = {'CPdId':C+Pd+Id,'C':C,'CPeIe':C+Pe+Ie,'CPd_dest':C+Pd[-int(len(Pd)/2):],'Cs':Cs,'CsPd'+str(int(len(Pd)/2))+'_dest':Cs+Pd[-int(len(Pd)/2):]} # para manter compatibilidade com o TrafficLabelling
    # f = 'CPd'+str(int(len(Pd)/2))+'Id'+str(int(len(Id)/2))

    rest_cols = comb_f[f]

elif dataset == 'TrafficLabelling/':
    # dataset original

    C = ['Protocol', 'FlowDuration', 'TotalFwdPackets', 'TotalBackwardPackets', 'TotalLengthofFwdPackets', 'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin', 'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax', 'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd', 'FlowBytes/s', 'FlowPackets/s', 'FlowIATMean', 'FlowIATStd', 'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd', 'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd', 'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'FwdURGFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets/s', 'BwdPackets/s', 'MinPacketLength', 'MaxPacketLength', 'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance', 'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount', 'ACKFlagCount', 'URGFlagCount', 'CWEFlagCount', 'ECEFlagCount', 'Down/UpRatio', 'AveragePacketSize', 'AvgFwdSegmentSize', 'AvgBwdSegmentSize', 'SubflowFwdPackets', 'SubflowFwdBytes', 'SubflowBwdPackets', 'SubflowBwdBytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward', 'ActiveMean', 'ActiveStd', 'ActiveMax', 'ActiveMin', 'IdleMean', 'IdleStd', 'IdleMax', 'IdleMin']

    Cs = ['Protocol', 'ACKFlagCount', 'ActiveMean', 'ActiveMin', 'AveragePacketSize', 'BwdIATMean', 'BwdPacketLengthMin', 'BwdPacketLengthStd',
                 'BwdPackets/s', 'FwdIATMean', 'FwdIATMin', 'FwdPacketLengthMean', 'FwdPackets/s', 'FwdPSHFlags', 'FlowDuration', 'FlowIATMean',
                 'FlowIATMin', 'FlowIATStd', 'Init_Win_bytes_backward', 'Init_Win_bytes_forward', 'PSHFlagCount', 'SubflowFwdBytes', 'SYNFlagCount',
                 'TotalLengthofFwdPackets']

    Pd = ['sPort0', 'sPort1', 'sPort2', 'sPort3', 'sPort4', 'sPort5', 'dPort0',
           'dPort1', 'dPort2', 'dPort3', 'dPort4', 'dPort5']

    Id = ['sIP0', 'sIP1', 'sIP2', 'sIP3', 'dIP0', 'dIP1', 'dIP2', 'dIP3']

    # Ie = dataset_clean.columns.str.match('(SourceIP)|(DestinationIP)')
    # Ie = dataset_clean.columns[Ie].to_list()
    Ie = ['SourceIP_DNS_int', 'SourceIP_Reverse_proxy', 'SourceIP_Web_Server_int', 'SourceIP_ext', 'DestinationIP_DNS_int', 'DestinationIP_Reverse_proxy', 'DestinationIP_Web_Server_int', 'DestinationIP_b_m_cast', 'DestinationIP_ext']
    # Pe = dataset_clean.columns.str.match('(SourcePort)|(DestinationPort)')
    # Pe = dataset_clean.columns[Pe].to_list()
    Pe = ['SourcePort_444', 'SourcePort_80', 'SourcePort_8080', 'SourcePort_dynamic', 'SourcePort_registered', 'SourcePort_system', 'DestinationPort_444', 'DestinationPort_80', 'DestinationPort_8080', 'DestinationPort_dynamic', 'DestinationPort_registered', 'DestinationPort_system']
    comb_f = {'CsPe':Cs+Pe,'CPe':C+Pe,'CsPd':Cs+Pd,'CPd':C+Pd,'PeIe':Pe+Ie,'PdIe':Pd+Ie,'PdId':Pd+Id,'CsPdIe':Cs+Pd+Ie,'CsPdId':Cs+Pd+Id,'CPdIe':C+Pd+Ie,'CPdId':C+Pd+Id,'C':C,'Cs':Cs,'CPeIe':C+Pe+Ie,'CPd_dest':C+Pd[-6:]}
    # f = sys.argv[2]
    rest_cols = comb_f[f]

    # dataset_clean = pd.read_csv(path + 'dataset_clean_exp_atrib.csv', index_col=[0])
    dataset_clean = pd.read_csv(dataset + 'dataset_clean_exp_atrib.csv', usecols=rest_cols+['label'])
    # dataset_clean = load_chunk('dataset_clean_exp_atrib.csv',s=20000)
    # dataset_clean = dataset_clean[rest_cols+['Label']]
    dataset_clean.rename(columns={'Label':'label'}, inplace=True)

labels = dataset_clean['label']
features = dataset_clean.loc[:, rest_cols].astype('float64')
del dataset_clean
LE = LabelEncoder()
LE.fit(labels)
labels = LE.transform(labels)

features_train_val, features_test, labels_train_val, labels_test = train_test_split(features, labels, test_size=.2, random_state=42, stratify=labels)

features_train, features_val, labels_train, labels_val = train_test_split(features_train_val, labels_train_val, test_size=.2, random_state=42, stratify=labels_train_val)

m_train = features_train.shape[0]
m_val = features_val.shape[0]
m_test = features_test.shape[0]
m = labels.shape[0]

scaler = StandardScaler()
scaler.fit(features_train)

features_train = pd.DataFrame(scaler.transform(features_train), index=features_train.index,columns=features_train.columns)
# features_train_val = scaler.transform(features_train_val)
features_val = pd.DataFrame(scaler.transform(features_val), index=features_val.index,columns=features_val.columns)
features_test = pd.DataFrame(scaler.transform(features_test), index=features_test.index,columns=features_test.columns)

del features_train_val
del labels_train_val



C:\Users\SE8ALUNO\Anaconda3\envs\myenv\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
features = {'train':features_train,'val':features_val, 'test':features_test}
labels = {'train':labels_train, 'val':labels_val, 'test':labels_test}
model_path = dataset+'saved_models/bin_'+f
model = tf.keras.models.load_model(model_path+'.h5',compile=False)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.trainable = False

# _, train_acc = model.evaluate(features_train, labels_train>0, verbose=0)
# _, val_acc = model.evaluate(features_val, labels_val>0, verbose=0)
# _, test_acc = model.evaluate(features_test, labels_test>0, verbose=0)
# # print('Acurácias (%):\nTeste: {:2.4f}'.format(100*test_acc))
# print('Acurácias (%):\nTreino: {:2.4f}\nValidação: {:2.4f}\nTeste: {:2.4f}'.format(100*train_acc,100*val_acc,100*test_acc))

def create_adversarial_pattern(input_sample):#, input_labels):
    loss_object = tf.keras.losses.binary_crossentropy
    input_label = tf.constant(1,dtype='float32',shape=(input_sample.shape[0],1))
    input_sample = tf.convert_to_tensor(input_sample)
    with tf.GradientTape() as tape:
        tape.watch(input_sample)
        prediction = model(input_sample)
        loss = loss_object(input_label, prediction)

    # Get the gradients of the loss w.r.t to the input sample.
    gradient = tape.gradient(loss, input_sample)
    # gradient = tape.gradient(prediction, input_sample)
    # Get the sign of the gradients to create the perturbation
    # signed_grad = tf.sign(gradient)
    prediction = tf.squeeze(prediction).numpy()

    return prediction, gradient.numpy()#, msc_p, msc_fp


for cj in ['train','val','test']:

    prediction, gradient = create_adversarial_pattern(features[cj].values)#, labels[cj])
    msc_p = prediction > 0.5
    msc_fp = msc_p & (labels[cj] == 0)
    if cj == 'train':
        if dataset+f == 'TrafficLabelling/CPdId':
            conf_limiar = prediction[msc_fp].max()
        else:
            conf_limiar = np.percentile(prediction[msc_fp],99, interpolation='higher')

        msc_vp = msc_p & (labels[cj] != 0)

        VP = gradient[msc_vp].mean(axis=0)# CPdId, CPdIe
        FP = gradient[msc_fp].mean(axis=0)# CPdId, CPdIe
        # VP = gradient[msc_vp & (prediction <= conf_limiar)].mean(axis=0)
        # FP = gradient[msc_fp & (prediction <= conf_limiar)].mean(axis=0)
        vp_fp = np.array([VP, FP])

        # Similaridade entre a média dos gradientes (VP e FP) no conjunto treino
        print('Similaridade:',float(cosine_similarity(VP[np.newaxis,:], FP[np.newaxis,:])))
        # Normas das médias dos gradientes (VP e FP) no conjunto treino
        norma_vp_fp = np.linalg.norm(vp_fp, axis=1)
        print('||VP|| = {}\n||FP|| = {}'.format(norma_vp_fp[0],norma_vp_fp[1]))
        dl = {}

    if modo == 'cos':
        dados = np.concatenate((prediction[msc_p,np.newaxis],cosine_similarity(gradient[msc_p], vp_fp), np.linalg.norm(gradient[msc_p],axis=1, keepdims=True),labels[cj][msc_p,np.newaxis] == 0), axis=1)
        dados = pd.DataFrame(dados, index=features[cj][msc_p].index, columns=['Confiança','cos(\u03B8)\u2207_VP','cos(\u03B8)\u2207_FP','||\u2207||','FP?'])

    elif modo == 'pca':
        dados = np.concatenate((prediction[msc_p,np.newaxis],gradient[msc_p], labels[cj][msc_p,np.newaxis] == 0), axis=1)
        dados = pd.DataFrame(dados, index=features[cj][msc_p].index, columns=['Confiança']+features[cj].columns.tolist()+['FP?'])

    elif modo == 'pca_9':
        if cj == 'train':
            from sklearn.decomposition import PCA
            n=9
            pca = PCA(n_components=n)
            pca.fit(gradient)

        gradient = pca.transform(gradient)
        dados = np.concatenate((prediction[msc_p,np.newaxis],gradient[msc_p], labels[cj][msc_p,np.newaxis] == 0), axis=1)
        dados = pd.DataFrame(dados, index=features[cj][msc_p].index, columns=['Confiança']+['pca'+str(i) for i in range(n)]+['FP?'])

    dados.sort_values('Confiança',ascending=False,inplace=True)
    dados = dados.astype({'FP?': 'bool'})

    dl[cj] = dados[dados['Confiança']<=conf_limiar]

    refin = 0
    cf_fp, cf_size = dl[cj]['FP?'].sum(), dl[cj].shape[0]
    # adv_fp, adv_size = (((dl[cj]['cos(\u03B8)\u2207_FP']-dl[cj]['cos(\u03B8)\u2207_VP'])>=refin) & dl[cj]['FP?']).sum(), ((dl[cj]['cos(\u03B8)\u2207_FP']-dl[cj]['cos(\u03B8)\u2207_VP'])>=refin).sum()
    # cf_idx = dl[cj][dl[cj]['FP?']].iloc[-adv_fp].name
    # cf_idx_size = cf_size - np.flatnonzero(dl[cj].index == cf_idx).squeeze()
    
    print('Usando somente a confiança: {:>6d} FP em {:>4d} exemplos ({:2.2f}%)'.format(cf_fp, cf_size, 100*cf_fp/cf_size))
    # print('Usando confiança + Aprox. Adv.: {:>2d} FP em {:>4d} exemplos ({:2.2f}%)'.format(adv_fp, adv_size, 100*adv_fp/adv_size))
    # print('A partir do menos confiável: {:>5d} FP em {:>4d} exemplos ({:2.2f}%)\n'.format(adv_fp, cf_idx_size, 100*adv_fp/cf_idx_size))
    

labels['train'] = dl['train']['FP?']
features['train'] = dl['train'].iloc[:, :-1]
# LE.fit(labels['train'])
# labels['train'] = LE.transform(labels['train'])
scaler.fit(features['train'])
features['train'] = scaler.transform(features['train'])
labels['val'] = dl['val']['FP?']
features['val'] = dl['val'].iloc[:, 0:-1]
# labels['val'] = LE.transform(labels['val'])
features['val'] = scaler.transform(features['val'])

labels['test'] = dl['test']['FP?']
features['test'] = dl['test'].iloc[:, 0:-1]
# labels['test'] = LE.transform(labels['test'])
features['test'] = scaler.transform(features['test'])

cj='test'
if (dataset == 'TrafficLabelling/') & (f in ['C','CPd_dest']):
    p = np.linspace(0,.25,11)
else:
    p = np.linspace(0,1,11)
fracoes = dl[cj].shape[0]*p
fracoes = fracoes.round().astype(int)[1:]
# print(p)
# print(fracoes)

tot_fp = msc_fp.sum()
dx = p[2]-p[1]
cj_analise = np.zeros(fracoes.shape,dtype='int')
for i,j in enumerate(fracoes):
    cj_analise[i] = dl[cj].iloc[-j:,-1].sum()
# print(cj_analise)




Similaridade: 0.9900385173421111
||VP|| = 0.012315348333560067
||FP|| = 1.0910840641851431
Usando somente a confiança:    611 FP em 7773 exemplos (7.86%)
Usando somente a confiança:    150 FP em 1940 exemplos (7.73%)
Usando somente a confiança:    216 FP em 2447 exemplos (8.83%)


In [7]:
print('\nFP total no dl: {:d} de {:d}\n\n'.format(cf_fp,tot_fp))
# print(np.column_stack((cj_analise, cj_analise/tot_fp)))
print('\nAUC: {:.3f} (sem 2ª rede)\n\n'.format(np.trapz(cj_analise/tot_fp, dx=dx)/((len(cj_analise)-1)*dx)))


rodadas = 20

if modo == 'pca_9':
    atbs = np.floor(np.linspace(0,dados.shape[1]-1,4)[1:]).astype(int)
else:
    atbs = [1, dados.shape[1]-1]# colocar 1 antes

for atb in atbs:
    cj_analise = np.zeros(fracoes.shape)

    # atb = dados.shape[1]-1 # 1 para apenas confinça, 4 para confiança + aprox adv
    # atb = 1 # 1 para apenas confinça, 4 para confiança + aprox adv

    if segunda == 'mlp':
        dl2 = np.zeros(labels[cj].shape)
        for n in range(rodadas):
            model2 = tf.keras.models.Sequential([tf.keras.layers.Dense(10, input_dim=features['train'][:,:atb].shape[1], activation='relu'), tf.keras.layers.Dense(10, activation='relu'), tf.keras.layers.Dense(1, activation='sigmoid')])

            model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
            es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, patience=5, restore_best_weights=True)
            model2.fit(features['train'][:,:atb], labels['train'], epochs=150, verbose=0, batch_size = 256, validation_data=(features['val'][:,:atb], labels['val']), callbacks=[es])

            dl2 += model2.predict(features[cj][:,:atb]).squeeze()

        dl2 = dl2/rodadas
        msg = ''

    elif segunda == 'knn':
        from sklearn.neighbors import KNeighborsClassifier
        fracoes_val = dl['val'].shape[0]*p
        fracoes_val = fracoes_val.round().astype(int)[1:]
        # cj_analise = np.zeros(fracoes.shape)
        aucs = np.zeros(rodadas)
        for l in range(1, rodadas):
            model2 = KNeighborsClassifier(n_neighbors=l)
            model2.fit(features['train'][:,:atb], labels['train'])
            dl2 = model2.predict_proba(features['val'][:,:atb])
            dl2 = pd.DataFrame(np.column_stack((dl2[:,1],labels['val'])),index=labels['val'].index,columns=['Conf2',labels['val'].name])
            dl2.sort_values(by=['Conf2'], inplace=True)
            for i,j in enumerate(fracoes_val):
                cj_analise[i] = dl2.iloc[-j:,-1].sum()
            aucs[l] = np.trapz(cj_analise/tot_fp, dx=dx)/((len(cj_analise)-1)*dx)

        n_neighbors=aucs.argmax()
        model2 = KNeighborsClassifier(n_neighbors=n_neighbors)
        model2.fit(features['train'][:,:atb], labels['train'])
        dl2 = model2.predict_proba(features['test'][:,:atb])[:,1]
        msg = ', neighbors={:d}'.format(n_neighbors)

    dl2 = pd.DataFrame(np.column_stack((dl2,labels[cj])),index=labels[cj].index,columns=['Conf2',labels[cj].name])
    dl2.sort_values(by=['Conf2'], inplace=True)
    for i,j in enumerate(fracoes):
        cj_analise[i] = dl2.iloc[-j:,-1].sum()
    
    # print('\natb: {:d}\nFP total no dl: {:d} de {:d}\n'.format(atb,cf_fp,tot_fp))
    print(np.column_stack((cj_analise, cj_analise/tot_fp)))
    print('\nAUC: {:.3f} (atb = {:d}{:s})\n'.format(np.trapz(cj_analise/tot_fp, dx=dx)/((len(cj_analise)-1)*dx), atb, msg))
    print('\nAUC: {:.3f} (val)\n\n'.format(aucs.max()))


FP total no dl: 216 de 219



AUC: 0.817 (sem 2ª rede)


[[ 63.           0.28767123]
 [116.           0.52968037]
 [157.           0.71689498]
 [180.           0.82191781]
 [197.           0.89954338]
 [198.           0.90410959]
 [199.           0.9086758 ]
 [210.           0.95890411]
 [214.           0.97716895]
 [216.           0.98630137]]

AUC: 0.817 (atb = 3, neighbors=19)


AUC: 0.567 (val)


[[ 69.           0.31506849]
 [116.           0.52968037]
 [170.           0.77625571]
 [191.           0.87214612]
 [204.           0.93150685]
 [205.           0.93607306]
 [206.           0.94063927]
 [212.           0.96803653]
 [213.           0.97260274]
 [216.           0.98630137]]

AUC: 0.842 (atb = 6, neighbors=19)


AUC: 0.579 (val)


[[ 66.           0.30136986]
 [120.           0.54794521]
 [162.           0.73972603]
 [188.           0.85844749]
 [202.           0.92237443]
 [203.           0.92694064]
 [203.           0.92694064]
 [212.           0.96803653]
 [213.        

In [20]:
dl2_path = 'C:/Users/SE8ALUNO/maj_ricardo/google/doutorado/duvidas/'+dataset+'adv/dl2_'+f+'.csv'

# dl2.rename(columns={'Conf2':'Conf2_adv','FP?':'FP?_adv'}, inplace=True)
# dl2.to_csv(dl2_path)
dl2 = pd.read_csv(dl2_path,index_col=0,float_precision='round_trip')

for i,j in enumerate(fracoes):
  cj_analise[i] = dl2.iloc[-j:,-1].sum()
print('\natb: {:d}\nFP total no dl: {:d} de {:d}\n'.format(atb,cf_fp,tot_fp))
print('AUC: {:.3f}\n'.format(np.trapz(cj_analise/tot_fp, dx=dx)/((len(cj_analise)-1)*dx)))
# print(np.column_stack((cj_analise, cj_analise/tot_fp)))


atb: 4
FP total no dl: 309 de 318

AUC: 0.818

